In [5]:
import heapq

def find_blank_tile(board):
    for i in range(3):
        for j in range(3):
            if board[i][j] == 0:
                return (i, j)
    return None

def get_possible_moves(blank_pos):
    x, y = blank_pos
    moves = []
    if x > 0: moves.append((x - 1, y))  # Up
    if x < 2: moves.append((x + 1, y))  # Down
    if y > 0: moves.append((x, y - 1))  # Left
    if y < 2: moves.append((x, y + 1))  # Right
    return moves

def swap(board, pos1, pos2):
    new_board = [row[:] for row in board]
    new_board[pos1[0]][pos1[1]], new_board[pos2[0]][pos2[1]] = new_board[pos2[0]][pos2[1]], new_board[pos1[0]][pos1[1]]
    return new_board

def calculate_h_manhattan(board, goal):
    distance = 0
    for i in range(3):
        for j in range(3):
            value = board[i][j]
            if value != 0:
                target_x, target_y = divmod(value - 1, 3)
                distance += abs(target_x - i) + abs(target_y - j)
    return distance

def a_star_manhattan(start, goal):
    open_set = []
    heapq.heappush(open_set, (0, start, []))  # (f, board, path)
    g_costs = {tuple(map(tuple, start)): 0}

    while open_set:
        current_cost, current_board, path = heapq.heappop(open_set)

        blank_pos = find_blank_tile(current_board)
        g = g_costs[tuple(map(tuple, current_board))]
        h = calculate_h_manhattan(current_board, goal)
        f = g + h

        # Print current state and costs
        print("Current state:")
        for row in current_board:
            print(row)
        print(f"H-value: {h}, G-value: {g}, F-value: {f}\n")

        if current_board == goal:
            return path

        for move in get_possible_moves(blank_pos):
            new_board = swap(current_board, blank_pos, move)
            g = g_costs[tuple(map(tuple, current_board))] + 1

            if tuple(map(tuple, new_board)) not in g_costs or g < g_costs[tuple(map(tuple, new_board))]:
                g_costs[tuple(map(tuple, new_board))] = g
                f = g + calculate_h_manhattan(new_board, goal)
                new_path = path + [new_board]
                heapq.heappush(open_set, (f, new_board, new_path))

    return None  # No solution found

def print_solution_path(path):
    for step in path:
        for row in step:
            print(row)
        print()

start_state = [[1, 2, 3], [4, 0, 5], [7, 8, 6]]
goal_state = [[1, 2, 3], [4, 5, 6], [7, 8, 0]]

solution_path_manhattan = a_star_manhattan(start_state, goal_state)
print("Solution path using Manhattan distance:")
if solution_path_manhattan is not None:
    print_solution_path(solution_path_manhattan)
else:
    print("No solution found.")


Current state:
[1, 2, 3]
[4, 0, 5]
[7, 8, 6]
H-value: 2, G-value: 0, F-value: 2

Current state:
[1, 2, 3]
[4, 5, 0]
[7, 8, 6]
H-value: 1, G-value: 1, F-value: 2

Current state:
[1, 2, 3]
[4, 5, 6]
[7, 8, 0]
H-value: 0, G-value: 2, F-value: 2

Solution path using Manhattan distance:
[1, 2, 3]
[4, 5, 0]
[7, 8, 6]

[1, 2, 3]
[4, 5, 6]
[7, 8, 0]



In [2]:
class Node:
    def __init__(self, data, level, fval):
        """ Initialize the node with the data, level of the node and the calculated fvalue """
        self.data = data
        self.level = level
        self.fval = fval

    def generate_child(self):
        """ Generate child nodes from the given node by moving the blank space
            either in the four directions {up, down, left, right} """
        x, y = self.find(self.data, 0)  # Looking for the blank tile represented by 0
        val_list = [[x, y - 1], [x, y + 1], [x - 1, y], [x + 1, y]]
        children = []
        for i in val_list:
            child = self.shuffle(self.data, x, y, i[0], i[1])
            if child is not None:
                child_node = Node(child, self.level + 1, 0)
                children.append(child_node)
        return children

    def shuffle(self, puz, x1, y1, x2, y2):
        """ Move the blank space in the given direction and if the position value are out
            of limits, return None """
        if 0 <= x2 < len(self.data) and 0 <= y2 < len(self.data):
            temp_puz = self.copy(puz)
            temp = temp_puz[x2][y2]
            temp_puz[x2][y2] = temp_puz[x1][y1]
            temp_puz[x1][y1] = temp
            return temp_puz
        else:
            return None

    def copy(self, root):
        """ Copy function to create a similar matrix of the given node"""
        return [row[:] for row in root]

    def find(self, puz, x):
        """ Specifically used to find the position of the blank space """
        for i in range(len(self.data)):
            for j in range(len(self.data)):
                if puz[i][j] == x:
                    return i, j


class Puzzle:
    def __init__(self, size):
        """ Initialize the puzzle size by the specified size, open and closed lists to empty """
        self.n = size
        self.open = []
        self.closed = []

    def accept(self):
        """ Accepts the puzzle from the user """
        puz = []
        for _ in range(self.n):
            temp = list(map(int, input().split(" ")))  # Convert input to integers
            puz.append(temp)
        return puz

    def f(self, start, goal):
        """ Heuristic Function to calculate heuristic value f(x) = h(x) + g(x) """
        return self.h(start.data, goal) + start.level

    def h(self, start, goal):
        distance = 0
        for i in range(self.n):
            for j in range(self.n):
                target_value = start[i][j]
                if target_value != 0 and target_value != goal[i][j]:
                    # Find target position
                    for m in range(self.n):
                        for n in range(self.n):
                            if goal[m][n] == target_value:
                                distance += abs(m - i) + abs(n - j)
        return distance

    def process(self):
        print("Enter the start state matrix (use 0 for the blank space):\n")
        start = self.accept()
        print("Enter the goal state matrix:\n")
        goal = self.accept()

        start = Node(start, 0, 0)
        start.fval = self.f(start, goal)
        self.open.append(start)

        while True:
            cur = self.open[0]

            print("\nCurrent Node Selected:")
            for i in cur.data:
                print(" ".join(map(str, i)))
            print("")

            if self.h(cur.data, goal) == 0:
                print("\nGoal reached!")
                break

            print("\nGenerating child nodes:")
            for child in cur.generate_child():
                child.fval = self.f(child, goal)
                self.open.append(child)

                print("\nChild Node:")
                for i in child.data:
                    print(" ".join(map(str, i)))
                print("H-value:", child.fval - child.level)
                print("Level:", child.level)
                print("F-value:", child.fval)
                print("")

            self.closed.append(cur)
            del self.open[0]

            self.open.sort(key=lambda x: x.fval)

# Run the puzzle solver
puz = Puzzle(3)
puz.process()


Enter the start state matrix (use 0 for the blank space):

1 2 3
4 0 5
7 8 6
Enter the goal state matrix:

1 2 3
4 5 6
7 8 0

Current Node Selected:
1 2 3
4 0 5
7 8 6


Generating child nodes:

Child Node:
1 2 3
0 4 5
7 8 6
H-value: 3
Level: 1
F-value: 4


Child Node:
1 2 3
4 5 0
7 8 6
H-value: 1
Level: 1
F-value: 2


Child Node:
1 0 3
4 2 5
7 8 6
H-value: 3
Level: 1
F-value: 4


Child Node:
1 2 3
4 8 5
7 0 6
H-value: 3
Level: 1
F-value: 4


Current Node Selected:
1 2 3
4 5 0
7 8 6


Generating child nodes:

Child Node:
1 2 3
4 0 5
7 8 6
H-value: 2
Level: 2
F-value: 4


Child Node:
1 2 0
4 5 3
7 8 6
H-value: 2
Level: 2
F-value: 4


Child Node:
1 2 3
4 5 6
7 8 0
H-value: 0
Level: 2
F-value: 2


Current Node Selected:
1 2 3
4 5 6
7 8 0


Goal reached!
